In [1]:
import os
print(os.getcwd())
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
from torch.nn import Softmax
from torch.utils.data import DataLoader

from notebooks.utils.load_ckpt import define_model

import json
import argparse
import time
from itertools import chain
from tqdm import tqdm


/home/hyeryungson/mucoco


In [2]:
BATCH_SIZE=64
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# 원본 데이터 로드
samples=pd.read_csv('./notebooks/results/test_mucoco+add_preds.csv')
# label이 1인 데이터만 사용
sample1=samples.loc[samples['toxicity']>0.5].copy()

In [7]:
# load trained model
ckpt_path='/home/hyeryungson/mucoco/models/models_balanced/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
# ckpt_path='/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, tokenizer = define_model(ckpt_path, output_attentions=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [5]:
# # 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
# tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256], ['<|endoftext|>'], 50257)

In [6]:
# # tokenizer에 mask token 추가
# SPECIAL_TOKENS = {"mask_token": "<mask>"}
# tokenizer.add_special_tokens(SPECIAL_TOKENS)

1

In [7]:
# # 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
# tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256, 50257], ['<|endoftext|>', '<mask>'], 50257)

In [8]:
# verify if the code is correct
test_sent = sample1['text'].tolist()[0:10]
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [9]:
# cls token이 따로 없는데 잘 학습이 된게 맞을까? -> 상관없다고 하심 (교수님)
tokenizer.decode(batch['input_ids'][0])

"Ha ha, HILLARY LOST. All you hillary fools need to be rounded up and put in prison along with your leader. You are all anti--American scum. Perhaps we should dump you in Mexico along with all the illegal trash you let in....if you love 'em so much, go live with 'em. But you are not welcome here, you hate filled Marxist morons.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoft

In [10]:
classifier=model

In [11]:
torch.cuda.empty_cache()
# forward
classifier_output = classifier.forward(
    batch["input_ids"].cuda(),
    attention_mask=batch["attention_mask"].cuda(),
)
torch.cuda.empty_cache()

In [12]:
# get attentions
attentions = classifier_output["attentions"]
# attention_mask에서 1의 개수를 셈
lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
# 보고자 하는 attention layer 만 가져옴
attentions = attentions[
    10 # originally 10
]

In [18]:
cls_attns = attentions.max(1)[0][:, 0]

In [22]:
locate_ixes=[]
for i, attn in enumerate(cls_attns):
    # attention_mask가 1인 곳 까지의 attention을 보고, start of sentence와 end of sentence에 해당하는 token을 제거하고, softmax를 취한다.
    # current_attn = attn[: lengths[i]][1:-1].softmax(-1)
    current_attn = attn[: lengths[i]].softmax(-1) # <- current tokenizer does not add <s> and </s> to the sentence.
    # 이 값의 평균을 구한다.
    avg_value = current_attn.view(-1).mean().item()
    # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다. (+1 because we skipped the first token)
    # top_masks = ((current_attn > avg_value).nonzero().view(-1)) + 1
    top_masks = ((current_attn > avg_value).nonzero().view(-1))
    torch.cuda.empty_cache()
    top_masks = top_masks.cpu().tolist()
    # attention 값이 평균보다 큰 토큰의 수가 6 또는 문장 전체 토큰 수의 1/3 보다 크면  
    if len(top_masks) > min((lengths[i] - 2) // 3, 6):
        # 그냥 attention 값 기준 top k 개 (k = 6 또는 토큰 수/3)를 뽑는다.
        top_masks = (
            # current_attn.topk(min((lengths[i] - 2) // 3, 6))[1] + 1
            current_attn.topk(min((lengths[i] - 2) // 3, 6))[1]
        )
        top_masks = top_masks.cpu().tolist()
    # 현재 문장의 input id를 가져온다.
    current_sent = batch["input_ids"][i][: lengths[i]]
    count = 0
    top_masks_final = []
    # top_masks에 뽑힌 index를 돌면서
    for index in top_masks:
        # mask해야 할 토큰이 and, of, or, so 에 해당하지 않으면
        if tokenizer.decode(current_sent[index]) not in [ ## maybe add more!
            " and",
            " of",
            " or",
            " so",
        ]:
            # token을 mask 한다.
            # current_sent[index] = mask_token
            top_masks_final.append(index)
            # count 수를 늘린다.
            count += 1
        else:
            # 만약에 and, of, or, so 에 해당하면 아무것도 하지 않는다.
            pass
    locate_ixes.append(top_masks_final)

In [23]:
locate_ixes

[[78, 75, 77, 79, 76, 80],
 [2, 0, 3, 1],
 [1, 2],
 [16, 0, 17, 10, 7, 8],
 [0, 2, 10, 13],
 [52, 19, 54, 17, 47, 53],
 [39, 40, 36, 34, 41, 30],
 [0, 13, 22, 25, 26, 27],
 [31, 30, 0, 29, 2, 28],
 [137, 135, 136, 133, 1, 0]]

In [ ]:
outputs = TargetEmbeddings(
                        embed_dim=primary_embed_dim,
                        embed_lut=embed_luts[0],
                        sent_length=sent_length,
                        batch_size=batch_size,
                        device=device,
                        st=args.st,
                        init_value=init_value,
                        random_init=args.init == "random",
                        sampling_strategy=args.sampling_strategy,
                        sampling_strategy_k=args.sampling_strategy_k,
                        embed_scales=embed_scales,
                        metric=args.metric,
                        same_embed=args.same_embeds,
                        final_bias=final_bias,
                        eos_token_id=primary_tokenizer.eos_token_id
                    )

In [10]:
_time = time.time()
print("Loading RoBERTa")
classifier = model
print(f"Loaded RoBERTa classifier in {time.time() - _time}s")

# mask_token에 해당하는 id 설정
mask_token = tokenizer.encode("<mask>", add_special_tokens=False)[0]
output_sents=dict()
print("generating masks")
print("Extracting SLOT tokens")
# mask 대상 텍스트 로드
file = sample1['text'].tolist()
# 레이어 별로 attention 값을 기준으로 mask 해보기 위해서 outer for-loop 추가
for layer_num in range(0, 12):
    print(layer_num)
    output_sents[layer_num] = []
    # 텍스트를 32개씩 batch로 처리할 예정임
    for i in tqdm(range(0, len(file), 32)):
        # get batch
        input_lines = file[i : i + 32]
        # tokenize
        batch = tokenizer(
            input_lines, padding=True, return_tensors="pt", truncation=True
        )
        torch.cuda.empty_cache()
        # forward
        classifier_output = classifier.forward(
            batch["input_ids"].cuda(),
            attention_mask=batch["attention_mask"].cuda(),
        )
        torch.cuda.empty_cache()
        # get attentions
        attentions = classifier_output["attentions"]
        # attention_mask에서 1의 개수를 셈
        lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
        # 보고자 하는 attention layer 만 가져옴
        attentions = attentions[
            layer_num # originally 10
        ]  # 10 is chosen because it is the magical layer number of the grand elves
        # attentions.max(1)[0]: axis 1 에 대해서 max 값을 가져온다. (sequence에 대해서 여러개의 attention heads 중에서 가장 큰 값을 가져온다.) -> [:, 0] 그리고 나서 cls token의 attention을 가져온다.
        cls_attns = attentions.max(1)[0][:, 0]
        # batch에 있는 각 example에 대해서
        for i, attn in enumerate(cls_attns):
            # attention_mask가 1인 곳 까지의 attention을 보고, start of sentence와 end of sentence에 해당하는 token을 제거하고, softmax를 취한다.
            # current_attn = attn[: lengths[i]][1:-1].softmax(-1)
            current_attn = attn[: lengths[i]].softmax(-1) # <- current tokenizer does not add <s> and </s> to the sentence.
            # 이 값의 평균을 구한다.
            avg_value = current_attn.view(-1).mean().item()
            # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다. (+1 because we skipped the first token)
            # top_masks = ((current_attn > avg_value).nonzero().view(-1)) + 1
            top_masks = ((current_attn > avg_value).nonzero().view(-1))
            torch.cuda.empty_cache()
            top_masks = top_masks.cpu().tolist()
            # attention 값이 평균보다 큰 토큰의 수가 6 또는 문장 전체 토큰 수의 1/3 보다 크면  
            if len(top_masks) > min((lengths[i] - 2) // 3, 6):
                # 그냥 attention 값 기준 top k 개 (k = 6 또는 토큰 수/3)를 뽑는다.
                top_masks = (
                    # current_attn.topk(min((lengths[i] - 2) // 3, 6))[1] + 1
                    current_attn.topk(min((lengths[i] - 2) // 3, 6))[1]
                )
                top_masks = top_masks.cpu().tolist()
            # 현재 문장의 input id를 가져온다.
            current_sent = batch["input_ids"][i][: lengths[i]]
            count = 0
            # top_masks에 뽑힌 index를 돌면서
            for index in top_masks:
                # mask해야 할 토큰이 and, of, or, so 에 해당하지 않으면
                if tokenizer.decode(current_sent[index]) not in [ ## maybe add more!
                    " and",
                    " of",
                    " or",
                    " so",
                ]:
                    # token을 mask 한다.
                    current_sent[index] = mask_token
                    # count 수를 늘린다.
                    count += 1
                else:
                    # 만약에 and, of, or, so 에 해당하면 아무것도 하지 않는다.
                    pass
            # sent = (
            #     tokenizer.decode(current_sent)[3:-4]
            #     .replace("<mask>", " <mask>")
            #     .strip()
            # )
            sent = (
                # masking을 마친 input id를 decode 한다.
                tokenizer.decode(current_sent)
                .replace("<mask>", " <mask>")
                .strip()
            )
            # mask 된 곳이 없으면, 마지막에 추가한다.
            if "<mask>" not in sent:
                sent = sent + " <mask>."
            
            # 처리된 문장을 output_sents에 추가한다.
            output_sents[layer_num].append(sent)

# with open(args.out_file + ".intermediate_sentences.txt", "w") as f:
#     f.write("\n".join(output_sents))

Loading RoBERTa
Loaded RoBERTa classifier in 0.000164031982421875s
generating masks
Extracting SLOT tokens
0


100%|██████████| 45/45 [00:10<00:00,  4.40it/s]


1


100%|██████████| 45/45 [00:08<00:00,  5.42it/s]


2


...

100%|██████████| 45/45 [00:08<00:00,  5.27it/s]


10


100%|██████████| 45/45 [00:08<00:00,  5.30it/s]


11


100%|██████████| 45/45 [00:08<00:00,  5.33it/s]


In [57]:
# sample1['attn_L11'] = output_sents
# sample1['attn_L12'] = output_sents

In [24]:
for i in range(0, 12):
    sample1[f"attn_L{i+1}"] = output_sents[i]

In [11]:
import joblib
# joblib.dump(sample1, "notebooks/results/attn_based_locate_balanced_230524.pkl")
joblib.dump(sample1, "notebooks/results/attn_based_locate_binary_230524.pkl")

['notebooks/results/attn_based_locate_binary_230524.pkl']

In [12]:
sample1.sort_values(by='toxicity', ascending=False).iloc[1, ]['text'], sample1.sort_values(by='toxicity', ascending=False).iloc[1, ]['attn_L11']

KeyError: 'attn_L11'

In [ ]:
for i,j in zip(sample1['text'].tolist()[:10], output_sents[10][:10]):
    print("[original]")
    print(i)
    print("[masked]")
    print(j)

[original]
Ha ha, HILLARY LOST. All you hillary fools need to be rounded up and put in prison along with your leader. You are all anti--American scum. Perhaps we should dump you in Mexico along with all the illegal trash you let in....if you love 'em so much, go live with 'em. But you are not welcome here, you hate filled Marxist morons.
[masked]
Ha ha, HILLARY LOST. All you hillary fools need to be rounded up and put in prison along with your leader. You are all anti--American scum. Perhaps we should dump you in Mexico along with all the illegal trash you let in....if you love 'em so much, go live with 'em. But you are not welcome here, <mask> <mask> <mask> <mask> <mask> <mask>.
[original]
Two dumbs makes one dead.  Happens all too frequently.
[masked]
<mask> <mask> <mask> <mask> one dead.  Happens all too frequently.
[original]
Trash site.  Just like Hillary Clinton.
[masked]
Tr <mask> <mask>.  Just like Hillary Clinton.
[original]
Chomp, chomp ... SLURP! Pigs at the trough. More slo